In [1]:
import sys
sys.path.insert(1, '../../../..')

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
from scipy import interpolate
import copy

from methods.PINN import PINN
from methods.methodsDataset.PINNDataset import PINNDataset
from methods.DataDrivenMethods import DDMethod
from solvers.PoissonSolver import PoissonSolver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': None, 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_PINN = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [4]:
def logn_fct(grid, num_fct, num_sample = 5, mean = 0., sigma = 1.):
    
    assert num_sample < len(grid)
    
    idx = np.arange(0, len(grid), int(len(grid)/num_sample) + 1)
    idx[-1] = len(grid) - 1
    
    x = grid[idx]
    
    U = []
    for _ in range(num_fct):
        
        u = np.random.lognormal(mean, sigma, size = num_sample)
        f = interpolate.interp1d(x, u, kind = 'quadratic')
        min_D = np.abs(np.min(np.minimum(f(grid), 0)))
        new_D = f(grid) + min_D + 1e-1
        U.append(new_D)
    
    return np.stack(U)

In [5]:
solver = PoissonSolver(params=params_solver)
VD = logn_fct(solver.x, num_fct=100)
VU = solver.Vsolve(vect = 'D', D = VD)

In [6]:
d_train, d_val, u_train, u_val = sklearn.model_selection.train_test_split(VD, VU, 
                                                                          test_size=0.2, 
                                                                          random_state=123)

In [7]:
nx = params_solver['nx']
D_train = torch.Tensor(d_train).view(-1, 1)
D_val = torch.Tensor(d_val).view(-1, 1)

In [8]:
x = torch.Tensor(solver.x).view(-1, 1)
X_train = x.repeat(d_train.shape[0], 1)
X_val = x.repeat(d_val.shape[0], 1)

In [9]:
DX_train = torch.cat((D_train, X_train), dim=1)
DX_val = torch.cat((D_val, X_val), dim=1)
U_train = torch.Tensor(u_train).view(-1, 1)
U_val = torch.Tensor(u_val).view(-1, 1)

In [10]:
def get_dataset():
    return DX_train, DX_val, U_train, U_val

In [11]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [12]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 2, 8)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 8, 120)
        layers += [out_features]
        
    layers += [output_size]
    params_PINN_trial = copy.deepcopy(params_PINN)
    params_PINN_trial['layer_dims'] = layers
    params_PINN_trial['activations'] = 'tanh'
    return PINN(params={'solver':params_solver, 'method':params_PINN_trial})

In [13]:
def objective(trial):
    # Generate the model.
    
    model = define_model(trial, 2, 1)
    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    
    DX_train, DX_val, U_train, U_val = get_dataset()
    
    torch.manual_seed(params_PINN['seed'])
    
    n_epochs = 20000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam'}
    # Training of the model.
    
    # w_r = trial.suggest_float("w_r", 1e-1, 1e+3, log=True)
    model.fit(hyperparameters = hyperparameters, 
              DX_train = DX_train, DX_val = DX_val, U_val = U_val, U_train = U_train,
              physics_ratio = 0.,
              data_ratio = 1.)
    
    V = model.loss_dict['val']
    # R = model.loss_dict['train']['residual']
    # B = model.loss_dict['train']['ic_bc']
    
    # S = [r + b for r, b in zip(R, B)]
    m = min(V)
    
    trial.report(m, n_epochs)

    return m

In [14]:
sampler = TPESampler(seed=params_PINN['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-21 16:47:11,197] A new study created in memory with name: no-name-57ce8e2a-a23b-47bc-bb26-e837c5cccb36


In [ ]:
study.optimize(objective, n_trials=10)

[tr : 1.3e-03, val : 1.7e-03]: 100%|█████| 20000/20000 [00:38<00:00, 518.54it/s]
[I 2023-08-21 16:47:50,500] Trial 0 finished with value: 0.0016470539849251509 and parameters: {'n_layers': 6, 'units_l0': 40, 'units_l1': 33, 'units_l2': 70, 'units_l3': 89, 'units_l4': 55, 'units_l5': 118, 'lr': 0.0011336766867014445}. Best is trial 0 with value: 0.0016470539849251509.
[tr : 1.3e-03, val : 1.7e-03]: 100%|█████| 20000/20000 [00:37<00:00, 529.07it/s]
[I 2023-08-21 16:48:28,307] Trial 1 finished with value: 0.0016645031282678246 and parameters: {'n_layers': 5, 'units_l0': 52, 'units_l1': 46, 'units_l2': 90, 'units_l3': 57, 'units_l4': 14, 'lr': 0.0001563625577729033}. Best is trial 0 with value: 0.0016470539849251509.
[tr : 1.3e-03, val : 1.7e-03]: 100%|█████| 20000/20000 [00:41<00:00, 478.62it/s]
[I 2023-08-21 16:49:10,098] Trial 2 finished with value: 0.0016083437949419022 and parameters: {'n_layers': 7, 'units_l0': 28, 'units_l1': 27, 'units_l2': 68, 'units_l3': 68, 'units_l4': 79, 'unit

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'w_r':
        w_r = value

Use this to fill config_step_1.py

In [ ]:
print(activations, '\n', layer_dims, '\n', lr, '\n')